# Fundamentals of Machine Learning - 2022
## Report 2 - Classifying with convnets
Pablo Chehade  
Última modificación: 25/09/2022


### Introducción

Hay que construir 2 modelos: uno con ML clásico y otro con NN. El problema es uno de clasificación de imágenes (supervisado)

### 

### Análisis Exploratorio de los Datos

### Data augmentation?

### MODELO 1: Random Forest

¿Por qué este modelo en particular? Corre rápido

### MODELO 2: DL

### Evaluación de test